In [38]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from dfply import *
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [39]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')

## Introduction
In this notebook, we calculate the monthly usage statistics for the maps posted online in the Gallery for Wasatch Front Regional Council. (https://wfrc.maps.arcgis.com/home/gallery.html?view=grid&sortOrder=asc&sortField=title) These maps can be categorized as Web Mapping Applications, Document Links, StoryMaps, or other types. ESRI saves the usage statistics for the Web Mapping Applications, whereas the Document Links store usage statistics elsewhere. Specifically, in this notebook we get the monthly usage statistics for a certain year for all the Web Mapping Applications.

## Connect to ArcGIS Online
In order to get the usage statistics for the WFRC gallery maps, we first need to connect to ArcGIS Pro Online. To do this, run the following command inserting your WFRC GIS username.

In [2]:
#gis = GIS("https://arcgis.com", "user_name")
gis = GIS("https://arcgis.com", "cday_wfrc")
print("connected")

connected


## Functions
Below we store the function used to calculate the monthly usage value for any given map. 

In [3]:
def get_monthly_usage(gismap, year):
    # get the last 12 months worth of usage data and convert to a pandas dataframe
    map = pd.DataFrame(gismap.usage("1Y")) 
    # create a map name columne, and divide up the date column to year, month, and day
    map['MapName'] = gismap.title
    map['Year'] = pd.DatetimeIndex(map['Date']).year
    map['Month'] = pd.DatetimeIndex(map['Date']).month
    map['Day'] = pd.DatetimeIndex(map['Date']).day
    # filter to the desired year, group by the month, and summarize all the usages of each month
    map = (map >>
        mask(X.Year == year) >>
        group_by(X.Month, X.Year, X.MapName) >>
        summarize(monthlyUsage = X.Usage.sum())
    )
    # return the monthly usage statistics for the given map
    return map

## Read in all ESRI Tracked Online ArcGIS Maps
Here we read in all the Web Mapping Applications on the WFRC Gallery online page. Each map is read in using their unique url id and the gis command.

In [34]:
map1 = gis.content.get('87827ba730d44a09aeeae83a8f9dc43e') # Active Transportation GIS Data Resources
map2 = gis.content.get('50e00e43d8eb4df082aedeb2aa2b94f0') # Bike Comfort Web App
map3 = gis.content.get('50820dd5d2914b7da80e417dafe49d52') # Data Resources for Coordinating Housing & Transportation
map4 = gis.content.get('5184fbb871094dfeb87671d81bdbd3ee') # Existing Conditions: Wasatch Front Region
map5 = gis.content.get('fce73f8dea5b418db18ce090ee069fe3') # Generalized Future Land Use Map
map6 = gis.content.get('d1a3faa471264ed19d37fd4181c1f827') # Southwest Salt Lake County - Transportation Solutions
map7 = gis.content.get('1d40a6dc0a644abc8ec84f81cebeb6b2') # Street Connectivity Analysis
map8 = gis.content.get('8c17c889bc7848c788dabe4cee4a30a6') # Taxable Sales Comparison - 2019 to 2020 by Quarter
map9 = gis.content.get('a07232349fe6443c9de33aab218e218f') # Transportation and Land Use Connection (TLC) Projects Map
map10 = gis.content.get('151ffeb028f24e7d835002824b630b63') # Transportation Improvement Program (TIP) Map
map11 = gis.content.get('e3e0428210b5474d8a889375981b03a5') # usRAP Safety Data Map
map12 = gis.content.get('253a6472ebae4bdcac9dc2632906ac56') # UTA First Mile / Last Mile
map13 = gis.content.get('7d1b1df5686c41b593d1e5ff5539d01a') # Walkability and Measuring Urban Street Design
map14 = gis.content.get('1491ec0512aa43f88d27eff4bc9c71e0') # Wasatch Bike Plan
map15 = gis.content.get('3132c740d147403d8257ad277dc000dc') # Wasatch Front Park and Trail Accessibility
map16 = gis.content.get('e27ec6aa6fa242ed8396fe8ac29a80c7') # WFRC Housing Inventory Explorer (Jan 2020)

## Calculate the Monthly Usage
Using the created function, go through each map and get the monthly usage statistics for 2022. 

In [35]:
statsmap1 = get_monthly_usage(map1, 2022)
statsmap2 = get_monthly_usage(map2, 2022)
statsmap3 = get_monthly_usage(map3, 2022)
statsmap4 = get_monthly_usage(map4, 2022)
statsmap5 = get_monthly_usage(map5, 2022)
statsmap6 = get_monthly_usage(map6, 2022)
statsmap7 = get_monthly_usage(map7, 2022)
statsmap8 = get_monthly_usage(map8, 2022)
statsmap9 = get_monthly_usage(map9, 2022)
statsmap10 = get_monthly_usage(map10, 2022)
statsmap11 = get_monthly_usage(map11, 2022)
statsmap12 = get_monthly_usage(map12, 2022)
statsmap13 = get_monthly_usage(map13, 2022)
statsmap14 = get_monthly_usage(map14, 2022)
statsmap15 = get_monthly_usage(map15, 2022)
statsmap16 = get_monthly_usage(map16, 2022)

## Join Map Usage Stats and Write CSV File
Bind all 16 map tables together to one large usage table and then write it to a csv file to the outputs folder

In [ ]:
stats = pd.concat([statsmap1,statsmap2,statsmap3,statsmap4,statsmap5,statsmap6,statsmap7,statsmap8,statsmap9,statsmap10,
                   statsmap11,statsmap12,statsmap13,statsmap14,statsmap15,statsmap16])
stats

In [37]:
stats.to_csv("Outputs/esriMapUsage.csv")

Helpful Links
- https://support.esri.com/en/technical-article/000021762
- https://developers.arcgis.com/python/guide/using-the-gis/
- https://wfrc.maps.arcgis.com/home/notebook/notebookhome.html#